In [ ]:
#https://ascopubs.org/action/doSearch?field1=AllField&text1=bevacizumab&Ppub=&Ppub=&AfterYear=2015&BeforeYear=2021&pageSize=1https://ascopubs.org/action/doSearch?field1=AllField&text1=bevacizumab&Ppub=&Ppub=&AfterYear=2015&BeforeYear=2021&pageSize=20'

# 0- Connect to colab drive

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
drive_dir = '/drive/MyDrive'
data_dir = drive_dir + '/nlp_nature'

# 1- Récupération des liens vers les pages des articles.

In [ ]:
! pip install beautifulsoup4==4.9.3

     |████████████████████████████████| 115 kB 7.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
jco_url = 'https://ascopubs.org/action/doSearch?field1=AllField&text1=bevacizumab&Ppub=&Ppub=&AfterYear=2015&BeforeYear=2021&pageSize=1https://ascopubs.org/action/doSearch?field1=AllField&text1=bevacizumab&Ppub=&Ppub=&AfterYear=2015&BeforeYear=2021&pageSize=20'
html_text = requests.get(jco_url).text

In [ ]:
soup = BeautifulSoup(html_text, 'html.parser')

In [ ]:
print(soup.title)

<title>[All: bevacizumab] AND [Publication Date: (01/01/2015 TO 12/31/2021)] :  Search</title>


## 1-1- Recuperation des hyperliens vers les différents articles

In [ ]:
href_prefix = 'https://ascopubs.org'

In [ ]:
links = []
attrs={'class':'ref nowrap'}
for link in soup.find_all('a', attrs=attrs):
  links.append(link.get('href'))

print(len(links))

20


In [ ]:
print(links[0])

/doi/full/10.1200/JCO.20.01225


## 1-2- Navigation sur les pages secondaires

In [ ]:
#Lecture des autres pages de liens si all_pages est à True

all_pages=True
if all_pages :
  page=2
  nb_links = 1
  while nb_links != 0 :
    url = jco_url + '&startPage=' + str(page) 
    # Lecture de la page
    soup_tmp = BeautifulSoup(requests.get(url).text, 'html.parser')
    articles_links_tmp = [ link.get('href') for link in soup_tmp.find_all('a', attrs=attrs)]
    
    nb_links = len(articles_links_tmp)
    if nb_links > 0 :
      links = [*links, *articles_links_tmp]
    #print('nb_links ', nb_links)
    page += 1

In [ ]:
# On remplace la recherche d'abstract par celle d'article.
full_links = [href_prefix + link.replace('/full/', '/abs/') for link in links]

## 1-3- Creation d'un dataframe contenant les liens

In [ ]:
# creation d'un csv contenant les liens
import pandas as pd
jco_link_df = pd.DataFrame(full_links, columns=['link'])

In [ ]:
jco_link_df.head()

,link
0,https://ascopubs.org/doi/abs/10.1200/JCO.20.01225
1,https://ascopubs.org/doi/abs/10.1200/JOP.2015....
2,https://ascopubs.org/doi/abs/10.1200/JCO.2015....
3,https://ascopubs.org/doi/abs/10.1200/JCO.19.03158
4,https://ascopubs.org/doi/abs/10.1200/JCO.2014....


# 2- Extraction des inforamtions des  différents articles et saisie dans le dataframe


In [ ]:
## Champ Titre:
# <h1 class="chaptertitle">Title</h1>
## Champ auteurs :
# <a class="entryAuthor" >Yu-Hsi Lin</a>
## Champ abstract :
# <div class="h1F1d-Abstract">
# <p>Inhibiting glycolysis remains an aspirational approach for the treatment of cancer.</p></div>


In [ ]:
! pip install tdqm

  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1323 sha256=cff9302f508a0b13d4ceb9e6ccaac952dd7e4ada6f5bd91e06ff41ca6a5eae70
  Stored in directory: /root/.cache/pip/wheels/c6/f0/d9/9fa5ff78c0f9d5a0a427bbbb4893c283520ddfccb885ea2205
Successfully built tdqm


In [ ]:
import pandas as pd
import time
from tqdm import tqdm
len_max=300
separateur='##'

start_0=time.time()

l_suppress_type = ['CORRESPONDENCE', 'CASE REPORTS', 'CASE REPORT', 'Case Reports','EDITORIALS', 'Editorials',
                   'COMMENTS AND CONTROVERSIES', 'Health Care Delivery', 'ASCO SPECIAL ARTICLES',
                   'REVIEW ARTICLES', 'ERRATA', 'ERRATUM', 'ART OF ONCOLOGY','EDITORIAL', 'COMMENTARY', 
                   'Special Series: Quality Care Symposium', 'STATISTICS IN BRIEF','REVIEW ARTICLES',
                   'Clinical Reviews', 'DIAGNOSIS IN ONCOLOGY', 'SPECIAL SERIES: ADVANCES IN THE MANAGEMENT OF GYNECOLOGIC CANCERS',
                   'Guideline Summaries', 'COMMENTARIES', 'CLINICAL REVIEWS', 'Commentary', 'LETTERS TO THE EDITOR'
                   ]
#l_suppress_fields = [ w + separateur for w in l_suppress_words]

l_authors = []
l_title = []
l_type = []
l_journal = []
l_conclusion = []
l_field = []
l_article = []

for i in tqdm(range(len(full_links[:]))) :# enumerate(tqdm(articles_links, total=len(articles_links))) :
  link = full_links[i]
  
  print(link)
  html_2_text=requests.get(link).text
  #print(html_2_text)
  soup_2 = BeautifulSoup(html_2_text, 'html.parser')



  # Recuperation titre
  title_field = soup_2.find('div', {'class':"publicationContentTitle"}).get_text()
  #print('title_field : ', title_field)
  l_title.append(title_field)

  # Recuperation type
  type_field = soup_2.find('div', {'class':"tocHeading"}).get_text(separator=separateur).split('\n')
  type_article = type_field[1].replace('##', '')
  field_article = type_field[2].replace('##', '') 
  l_type.append(type_article)
  print(type_article)
  l_field.append(field_article)

  # Recuperation journal
  journal_field = soup_2.find('cite').get_text()
  #print('journal_field : ', journal_field)
  l_journal.append(journal_field)

  # Recuperation auteurs
  l_auth = []
  for l in soup_2.find_all('a', attrs={'class':'entryAuthor'}):
    l_auth.append(l.get_text())
  authors_field = ','.join(l_auth)
  #print('auteurs : ', authors_field)
  l_authors.append(authors_field)
  
  # Recuperation abstract
  raw_article = soup_2.find('article', class_="article")
  if raw_article != None :
     article_field = raw_article.get_text(separator=separateur)
  else :
     article_field = 'no article'
  l_article.append(article_field)

  raw_conclusion = soup_2.find('div', class_="abstractSection")
  if raw_conclusion != None :
     raw_conclusion_field = raw_conclusion.get_text(separator=separateur)
     l_heads = []
     for l in raw_conclusion.find_all('div', {'class':'sectionInfo'}) :
        text = l.get_text()
        #print(text)
        if 'conclusion' in text.lower() :
           conclusion_field = raw_conclusion_field.split(f'##{text}##')[-1]
     l_conclusion.append(conclusion_field.replace('##',' '))
  else :
     l_conclusion.append('no conclusion')
  #print('abstract : ', conclusion_field.replace('##',''))
  

#print(l_article)
#print(l_conclusion)

jco_link_df['title'] = l_title
jco_link_df['journal'] = l_journal
jco_link_df['authors'] = l_authors
jco_link_df['type'] = l_type
jco_link_df['field'] = l_field
jco_link_df['article'] = l_article
jco_link_df['conclusion'] = l_conclusion



end_0= time.time()

print('duree : ', end_0 - start_0)

  0%|          | 0/684 [00:00<?, ?it/s]

https://ascopubs.org/doi/abs/10.1200/JCO.20.01225


  0%|          | 1/684 [00:00<04:26,  2.56it/s]

ORIGINAL REPORTS
https://ascopubs.org/doi/abs/10.1200/JOP.2015.004218


  0%|          | 2/684 [00:01<07:10,  1.59it/s]

...

ORIGINAL REPORT
https://ascopubs.org/doi/abs/10.1200/JCO.2014.56.2959


100%|█████████▉| 683/684 [08:53<00:00,  1.17it/s]

ORIGINAL REPORTS
https://ascopubs.org/doi/abs/10.1200/JCO.2015.61.1509


100%|██████████| 684/684 [08:54<00:00,  1.28it/s]

REVIEW ARTICLES
duree :  534.188798904419


In [ ]:
jco_link_df.head()

,link,title,journal,authors,type,field,article,conclusion
0,https://ascopubs.org/doi/abs/10.1200/JCO.20.01225,\n\n \n \n\n ...,Journal of Clinical Oncology,"Chiara Cremolini,Search for articles by this a...",ORIGINAL REPORTS,Gastrointestinal Cancer,\n##Abstract##PURPOSE##A proper estimation of ...,FOLFOXIRI + bevacizumab significantly and mean...
1,https://ascopubs.org/doi/abs/10.1200/JOP.2015....,\n\n \n \n\n ...,Journal of Oncology Practice,"Stacie B. Dusetzina,Search for articles by thi...",Health Policy,Original Contributions,\n##Abstract##Section:##Choose##Top of page##A...,FOLFOXIRI + bevacizumab significantly and mean...
2,https://ascopubs.org/doi/abs/10.1200/JCO.2015....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Bradley J. Monk,Search for articles by this au...",ORIGINAL REPORTS,Gynecologic Cancer,\n##Abstract##Section:##Choose##Top of page##A...,"On the basis of the PFS, overall response rate..."
3,https://ascopubs.org/doi/abs/10.1200/JCO.19.03158,\n\n \n \n\n ...,Journal of Clinical Oncology,"Martin Reck,Search for articles by this author...",ORIGINAL REPORTS,Thoracic Oncology,\n##Abstract##Section:##Choose##Top of page##A...,ABCP seems tolerable and manageable versus ACP...
4,https://ascopubs.org/doi/abs/10.1200/JCO.2014....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Miguel Martín,Search for articles by this auth...",ORIGINAL REPORTS,Breast Cancer,\n##Abstract##Section:##Choose##Top of page##A...,The addition of bevacizumab to ET in first-l...


## 2-1 Sauvergarde de l'ensemble des articles

In [ ]:
jco_link_df.to_csv(data_dir + '/jco_link_df_bevacizumab_29092021.csv')

## 2-2- Sauvegarde des articles contenant une conclusion

In [ ]:
jco_conclusion_df = jco_link_df.loc[jco_link_df['conclusion'] != 'no conclusion']

In [ ]:
jco_link_df.shape

(684, 8)

In [ ]:
jco_conclusion_df.shape

(453, 8)

In [ ]:
jco_conclusion_df.to_csv(data_dir + '/jco_conclusion_df_bevacizumab_29092021.csv')